In [2]:

import pandas as pd
import os,time
import numpy as np
from tqdm import tqdm_notebook as tqdm


import scipy
from scipy import sparse
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
from sklearn import svm

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

### Utility functions for accuracy calculation etc.

In [3]:

def accuracy_score(predicted, real):
    assert(len(predicted) == len(real))
    matches=0
    for index in range(0,len(predicted)):
        if(real[index] == predicted[index]):
            matches += 1
    return matches/len(predicted)

def timing(f):
    def timer(*args, **kwargs):
        start = time.time()
        ret = f(*args, **kwargs)
        finish = time.time()
        print('%s took %0.3f seconds' % (f.__name__, (finish-start)))
        return ret
    return timer

%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


### Simultaneous Multiclass SVM Algorithm

In [4]:
class SMCSVM:
    def __init__(self,C=10,kernel='rbf',degree=1,sigma=1.0):
        self.kernel = kernel
        self.C=C
        self.degree = degree
        self.sigma=sigma
        self.alphas2=None
        self.fullXmatrix=None
        self.IntermXmatrix=None
        self.train_X_data = None
        self.num_classes = None
        
    def fit(self, X_data, Y_label):
        #self.kernel = kernel
        #self.C= C
        #self.degree = degree
        #self.sigma= sigma 
        self.train_X_data = X_data
        X = X_data
        trainLabelVector = Y_label
        num_classes = len(np.unique(Y_label))
        self.num_classes = num_classes
        
        fullXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes*((X_data.shape[1])+1)))

        count5 = 0
        
        print('\n Calculating fullXmatrix')
        for i in tqdm(range(0,X.shape[0])):
        
            for p in range(0,num_classes):
                if(trainLabelVector[i]!=p):   
                    for j in range(0,num_classes):
                        
                        if(j==trainLabelVector[i]):
                            fullXmatrix[count5,(j*(X_data.shape[1]+1)):((j+1)*(X_data.shape[1]+1))-1] = X[i,:]
                            fullXmatrix[count5,((j+1)*(X_data.shape[1]+1))-1] = 1.0
                        elif(j==p):
                            fullXmatrix[count5,(j*(X_data.shape[1]+1)):((j+1)*(X_data.shape[1]+1))-1] = -1*X[i,:]
                            fullXmatrix[count5,((j+1)*(X_data.shape[1]+1))-1] = -1.0
                            
                    count5 += 1
                        
                    
        m = fullXmatrix.shape[0]
        M1 = sparse.csr_matrix(fullXmatrix) 
        IntermXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes))
        
        if(kernel=='linear'):
            H = sparse.csr_matrix.dot(M1,M1.T)
        else:
            H,IntermXmatrix = self.getKernelizedProduct(X_data,Y_label,kernel,sigma, degree)        

        from cvxopt import matrix as cvxopt_matrix
        from cvxopt import solvers as cvxopt_solvers
        
        if(kernel=='linear'):
            P = cvxopt_matrix(H.todense())
        else:
            P = cvxopt_matrix(H)
        q = cvxopt_matrix(-1*np.ones((m, 1)))
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
        opts = {'maxiters' : 30,'solver':'mosek','show_progress':False}
        sol = cvxopt_solvers.qp(P, q, G, h ,options = opts)
        alphas2 = np.array(sol['x']) 
        self.alphas2=alphas2
        self.fullXmatrix=fullXmatrix
        self.IntermXmatrix=IntermXmatrix
   
    def predict(self,val_X_data):
        num_classes = self.num_classes
        train_X_data = self.train_X_data
        kernel = self.kernel
        degree = self.degree
        sigma = self.sigma
        alphas2=self.alphas2
        fullXmatrix=self.fullXmatrix
        IntermXmatrix=self.IntermXmatrix

        predictedLabelVector = []
        
        if(kernel == 'linear'):
            w = fullXmatrix.T@alphas2
            wForEachLabel = {}
            for i in range(0,num_classes):
                wForEachLabel[i] = np.array_split(w,num_classes)[i]
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                scoresForEachLabel = []
                for j in range(0,len(wForEachLabel)):
                    score = (wForEachLabel[j].T@np.append(val_X_data[i,:],1.0).reshape(-1,1))
                    scoresForEachLabel.append(score)
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
                
        elif(kernel == 'polynomial'):
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                #tempMatrix1 = np.zeros((num_classes*(val_X_data.shape[1]+1),num_classes))
                #for j in range(0,num_classes):
                #    tempMatrix1[j*(val_X_data.shape[1]+1):(j+1)*(val_X_data.shape[1]+1),j] = np.append(val_X_data[i,:],1.0)
                #scoresForEachLabel = (alphas2.T)@((1+(fullXmatrix@tempMatrix1))**degree)
                scoresForEachLabel = (alphas2.T)@(self.getKernelizedProductforPrediction(train_X_data,val_X_data,IntermXmatrix,i,num_classes,kernel,sigma,degree))
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
        elif(kernel == 'rbf'):
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                scoresForEachLabel = (alphas2.T)@(self.getKernelizedProductforPrediction(train_X_data,val_X_data,IntermXmatrix,i,num_classes,kernel,sigma,degree))
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
        return predictedLabelVector
    
    def getKernelizedProductforPrediction(self,X_data,val_X_data,IntermXmatrix,currentValIndex,num_classes,kernel,sigma,degree):
        X = X_data
        m = (X_data.shape[0]*(num_classes-1))
        H = np.zeros((m,num_classes))    
        matrix2 = (currentValIndex+1)*np.eye(num_classes)
        tempCoeff = (1/(X.shape[1]+1))
        
        if(kernel == 'polynomial'): 
            for i in (range(0,m)):
                for j in range(0,num_classes):
                    count1 = 0
                    for k in range(0,num_classes):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            if(temp3>0):
                                count1+=1
                                if(temp4>0):                             
                                    H[i,j] += (1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(val_X_data[temp4-1,:],1.0))))**degree
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += (1+((1/tempCoeff)*np.dot(np.append(val_X_data[temp4-1,:],1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree
        

        elif(kernel == 'rbf'): 
            for i in (range(0,m)):
                for j in range(0,num_classes):
                    count1 = 0
                    for k in range(0,num_classes):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            if(temp3>0):
                                count1+=1
                                if(temp4>0):                             
                                    H[i,j] += np.exp(-1*(np.linalg.norm(X[temp3-1,:]-val_X_data[temp4-1,:])**2)/(2*sigma**2))                        
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += np.exp(-1*(np.linalg.norm(np.append(val_X_data[temp4-1,:],1.0)-np.append(-1*X[-temp3-1,:],-1.0))**2)/(2*sigma**2))                    

        return H      
    
    def getKernelizedProduct(self,X_data,Y_label,kernel,sigma, degree):
        X = X_data
        trainLabelVector = Y_label
        num_classes = len(np.unique(Y_label))
        IntermXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes))
        count6 = 0
        m = (X_data.shape[0]*(num_classes-1))
        nonZeroColumnIndicesAndValuesInIntermXmatrix = []
        
        print('\n Calculating Kernalized Product')
        for i in tqdm(range(0,X.shape[0])):
            for p in range(0,num_classes):
                if(trainLabelVector[i]!=p):   
                    tempArray = []
                    for j in range(0,num_classes):
                        if(j==trainLabelVector[i]):
                            tempArray.append(j)
                            tempArray.append(i+1)
                            IntermXmatrix[count6,j] = i+1
                        elif(j==p):
                            tempArray.append(j)
                            tempArray.append(-(i+1))
                            IntermXmatrix[count6,j] = -(i+1)
                    count6 += 1
                    nonZeroColumnIndicesAndValuesInIntermXmatrix.append(tempArray)
            
        matrix2 = IntermXmatrix.T
        M2 = scipy.sparse.coo_matrix(IntermXmatrix)
        M3 = scipy.sparse.coo_matrix(IntermXmatrix.T)
        m = (X_data.shape[0]*(num_classes-1))
        H = np.zeros((m,m))
        tempCoeff = (1/(X.shape[1]+1))

        if(kernel == 'rbf'):
            matrix2 = matrix2.tolist()
            #X = X.tolist()
            print('\n Calculating Kernalized Product')
            for i in tqdm(range(0,m)):
                for j in range(0,i+1):
                    #nonZeroColumnIndicesAndValuesInIntermXmatrix[i][0]
                    #nonZeroIndicesLocation = np.nonzero(IntermXmatrix[i,:])
                    #temp8 = IntermXmatrix[i,:][nonZeroIndicesLocation][0]
                    temp8 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][1]
                    #temp9 = IntermXmatrix[i,:][nonZeroIndicesLocation][1]
                    temp9 =nonZeroColumnIndicesAndValuesInIntermXmatrix[i][3]
                    #locOftemp8 = nonZeroIndicesLocation[0]
                    locOftemp8 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][0]
                    #locOftemp9 = nonZeroIndicesLocation[1]
                    locOftemp9 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][2]
                    temp10 = matrix2[locOftemp8][j]
                    temp11 = matrix2[locOftemp9][j]
                    #temp10zero = 1
                    #temp11zero = 1
                    #if(temp10 == 0):
                        #temp10zero = 0
                    #if(temp11 == 0):
                        #temp11zero = 0
                    temp8Pos = 1
                    temp9Pos = 1
                    temp10Pos = 1
                    temp11Pos = 1
                    
                    if(temp8 < 0):
                        temp8Pos = -1
                    if(temp9 < 0):
                        temp9Pos = -1
                    if(temp10 < 0):
                        temp10Pos = -1
                    if(temp11 < 0):
                        temp11Pos = -1
                        
                    if(temp10 != 0):
                        #H[i,j] += (2.718281828459045)**(-1*normsquare(np.append(temp8Pos*X[int(temp8Pos*temp8)-1,:],temp8Pos),np.append(temp10Pos*X[int(temp10Pos*temp10)-1,:],temp10Pos))/(2*sigma**2))                       
                        H[i,j] += (2.718281828459045)**(-1*(np.linalg.norm(np.append(temp8Pos*X[int(temp8Pos*temp8)-1,:],temp8Pos)-np.append(temp10Pos*X[int(temp10Pos*temp10)-1,:],temp10Pos))**2)/(2*sigma**2))
                    if(temp11 != 0):
                        #H[i,j] += (2.718281828459045)**(-1*normsquare(np.append(temp9Pos*X[int(temp9Pos*temp9)-1,:],temp9Pos),np.append(temp11Pos*X[int(temp11Pos*temp11)-1,:],temp11Pos))/(2*sigma**2))
                        H[i,j] += (2.718281828459045)** (-1*(np.linalg.norm(np.append(temp9Pos*X[int(temp9Pos*temp9)-1,:],temp9Pos)-np.append(temp11Pos*X[int(temp11Pos*temp11)-1,:],temp11Pos))**2)/(2*sigma**2))    

        
        elif(kernel == 'polynomial'):
            print('\n Calculating Kernalized Product')
            for i in tqdm(range(0,m)):
                for j in range(0,i+1):
                    count1 = 0
                    #count2 = 0
                    for k in range(0,IntermXmatrix.shape[1]):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            #print(temp3,temp4)
                            if(temp3>0):
                                count1+=1
                                if(temp4>0): 
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(X[temp4-1,:],1.0))))**degree)
                                elif(temp4<0):                                    
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(-1*X[-temp4-1,:],-1.0))))**degree)
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp4-1,:],1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree)
                                elif(temp4<0):
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(-1*X[-temp4-1,:],-1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree)
              

        print('\n Calculating Kernalized Product')
        
        for i in tqdm(range(0,m)):
            for j in range(0,i+1):
                if(j>i):
                    H[i,j]=H[j,i]                 

        return H,IntermXmatrix

### Plotting Function

In [5]:
def plot_decision_regions(classifier,train_X_data,train_Y_data,resolution=1):
    
   # setup marker generator and color map
   markers = ('s', 'x', 'o', '^', 'v')
   colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
   X = train_X_data
   y = train_Y_data
   cmap = ListedColormap(colors[:len(np.unique(y))])
    
   # plot the decision surface
   x1_min, x1_max = X[:, 0].min() - 1 , X[:, 0].max() + 1
   x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
   xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
   np.arange(x2_min, x2_max, resolution))
   Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
   Z = np.array(Z) 
   Z = Z.reshape(xx1.shape)
   plt.contourf(xx1, xx2, Z, alpha=0.1, cmap=cmap)
   plt.xlim(xx1.min(), xx1.max())
   plt.ylim(xx2.min(), xx2.max())
   

   # plot all samples
   #X_test, y_test = X[test_idx, :], y[test_idx]
   for idx, cl in enumerate(np.unique(y)):
      plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
               alpha=0.8, c=cmap(idx),
               marker=markers[idx], label=cl)
   plt.legend(loc='upper left')
   plt.show()

### Cross validation with Bias variance Estimation Plot

In [6]:

def performCrossValidation(X1,Y1,num_folds,classifier):
    print('-------------Performing %d-fold cross validation------------------ '%(num_folds))
    fullXmatrix1 = X1
    fullYvector1 = Y1
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=num_folds)
    kf.get_n_splits(fullXmatrix1)
    
    ValErrorForEachFold = []
    ValAccuForEachFold=[]
    fold = 0
    for train_index, test_index in kf.split(fullXmatrix1):
        
        fold += 1
        
        X_train, X_test = fullXmatrix1[train_index], fullXmatrix1[test_index]
        y_train, y_test = fullYvector1[train_index], fullYvector1[test_index]
        
        classifier.fit(X_train, y_train)
        predictedLabels = classifier.predict(X_test)
        
        accuracy = accuracy_score(predictedLabels,y_test)
        error = 1- accuracy
        ValErrorForEachFold.append(error)
        ValAccuForEachFold.append(accuracy)
        
    mean_accuracy = np.mean(ValAccuForEachFold)
    mean_error = np.mean(ValErrorForEachFold)
    
    return mean_error, mean_accuracy

### Testing the Algorithm on Iris Dataset

In [6]:
dataset = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size = 0.2, random_state = 42)

In [7]:
kernel = 'rbf'
degree = 2
sigma = 0.2
C = 1
gamma = 1/(2*sigma**2)

In [8]:
clf = SMCSVM(C,kernel,degree,sigma)
%memit clf.fit(X_train,y_train)
predictedLabels = clf.predict(X_test)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),y_test.reshape(-1,1).ravel())
conMatrix


 Calculating fullXmatrix




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating Kernalized Product



peak memory: 137.20 MiB, increment: 5.40 MiB

 Calculating prediction values


col_0,0,1,2
row_0,,,
0,10,0,0
1,0,9,0
2,0,0,11


In [9]:
clf = svm.SVC(C,gamma=gamma, decision_function_shape='ovo',kernel=kernel,degree=degree)
%memit clf.fit(X_train, y_train) 
predictedLabels=clf.predict(X_test)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),y_test.reshape(-1,1).ravel())
conMatrix

peak memory: 140.39 MiB, increment: 1.17 MiB


col_0,0,1,2
row_0,,,
0,10,0,0
1,0,9,0
2,0,0,11


In [10]:
clf = svm.LinearSVC(C=C,multi_class='ovr',max_iter=10**5)
%memit clf.fit(X_train, y_train) 
predictedLabelVector = clf.predict(X_test)
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),y_test.reshape(-1,1).ravel())
conMatrix

peak memory: 140.45 MiB, increment: 0.00 MiB


col_0,0,1,2
row_0,,,
0,10,0,0
1,0,9,0
2,0,0,11


### Testing the Algorithm on Wine Dataset

In [11]:
dataset = datasets.load_wine()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size = 0.2, random_state = 42)

In [12]:
kernel = 'rbf'
degree = 1
sigma = 10
C = 10
gamma = 1/(2*(sigma**2))

In [13]:
clf = SMCSVM(C,kernel,degree,sigma)
clf.fit(X_train,y_train)
predictedLabels = clf.predict(X_test)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),y_test.reshape(-1,1).ravel())
conMatrix


 Calculating fullXmatrix




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating prediction values


col_0,0,1,2
row_0,,,
0,12,0,1
1,0,11,2
2,2,3,5


In [14]:
clf = svm.SVC(C,gamma=gamma, decision_function_shape='ovo',kernel='rbf')
clf.fit(X_train, y_train) 

predictedLabels=clf.predict(X_test)

conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),y_test.reshape(-1,1).ravel())
conMatrix

col_0,0,1,2
row_0,,,
0,10,3,2
1,3,11,2
2,1,0,4


In [15]:
clf = svm.LinearSVC(C=C, multi_class='ovr',max_iter=10**5)
clf.fit(X_train, y_train) 
predictedLabelVector = clf.predict(X_test)
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),y_test.reshape(-1,1).ravel())
conMatrix

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


col_0,0,1,2
row_0,,,
0,13,0,0
1,1,14,0
2,0,0,8


### Testing  with different datasets with different Kernels, C and Sigma 

In [7]:
pt = PrettyTable()
pt.title = 'Comparison of Error Rates and Execution times'
columns = ['Kernel', 'C', 'Sigma', 'Degree (if Poly. Kernel)', '1-v-r', 'Exec. time 1vR ' ,'1-v-1', 'Exec. time 1v1', 'SMCSVM','Exec. time SMCSVM']
pt.field_names = columns

pt1 = PrettyTable()
columns = ['Dataset', 'Number of Points', 'Number of Attributes', 'Number of Classes']
pt1.field_names = columns

In [8]:
def performtestonDataset(dataset, C, kernel, sigma, degree):
  
  gamma=1/(2*sigma**2)
  
  X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size = 0.2, random_state = 42)
  
  num_folds =4
  
  
  kernel_alias = kernel
  if kernel == 'poly' :
    kernel_alias='polynomial'
    
  start = time.time()
  clf = SMCSVM(C,kernel_alias,degree,sigma)
  mean_error_SMCSVM, mean_accuracy_SMCSVM= performCrossValidation(dataset.data,dataset.target,num_folds,clf)
  time_SMCSVM=time.time()-start
  del clf
  
  
   
  start = time.time()
  clf = svm.SVC(C,gamma=gamma, decision_function_shape='ovo',kernel=kernel, degree=degree)
  mean_error_OVO, mean_accuracy_OVO = performCrossValidation(dataset.data,dataset.target,num_folds,clf)
  time_OVO=time.time()-start
  del clf
  
  
  start = time.time()
  clf = svm.LinearSVC(C=C,multi_class='ovr',max_iter=10**5)
  clf.kernel=kernel
  mean_error_OVR, mean_accuracy_OVR = performCrossValidation(dataset.data,dataset.target,num_folds,clf)
  time_OVR=time.time()-start
  del clf
  
  if kernel == 'rbf':
    metrics = [kernel, C, sigma, 'NA', round(mean_error_OVR,4), round(time_OVR,4), round(mean_error_OVO,4), round(time_OVO,4), round(mean_error_SMCSVM,4), round(time_SMCSVM,4) ]
  if kernel =='linear':
    metrics = [kernel, C, 'NA', 'NA', round(mean_error_OVR,4), round(time_OVR,4), round(mean_error_OVO,4), round(time_OVO,4), round(mean_error_SMCSVM,4), round(time_SMCSVM,4) ]
  if kernel == 'poly':
    metrics = [kernel, C, 'NA', degree, round(mean_error_OVR,4), round(time_OVR,4), round(mean_error_OVO,4), round(time_OVO,4), round(mean_error_SMCSVM,4), round(time_SMCSVM,4)  ]
    
  return metrics
  

In [9]:
dataSets=[datasets.load_iris(),datasets.load_wine()]
kernel_params=['rbf', 'poly', 'linear']
C_params = [1,10,50]
degree_params =[2,3,4]
sigma_params=[0.2,0.7,1.2]

In [ ]:
count=0
decription=[]
#pt.clear_rows()
for dataSet in dataSets:
  pt1.clear_rows()
  if 'Iris' in dataSet.DESCR:
    description=['Iris', dataSet.data.shape[0], dataSet.data.shape[1], len(dataSet.target_names)]
  elif 'Wine' in dataSet.DESCR:
    description=['Wine', dataSet.data.shape[0], dataset.data.shape[1], len(dataSet.target_names)]
  elif 'Breast' in dataSet.DESCR:
    description=['Breast-Cancer', dataSet.data.shape[0], dataSet.data.shape[1], len(dataSet.target_names)]
  pt1.add_row(description)
  pt.clear_rows()
  for kernel in kernel_params:
    for C in C_params:
      for degree in degree_params:
        for sigma in sigma_params:
          metrics = performtestonDataset(dataSet,C, kernel, sigma, degree)
          pt.add_row(metrics)
          
  path = './test_results.txt'
  with open(path, 'a+') as thefile:
        thefile.write('\n')
        thefile.write(pt1.get_string())
        thefile.write('\n')
        thefile.write(pt.get_string())
        count+=1
        if count == len(dataSets):
            continue
  

-------------Performing 4-fold cross validation------------------ 

 Calculating fullXmatrix




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating prediction values


...



 Calculating fullXmatrix




 Calculating Kernalized Product




 Calculating Kernalized Product




 Calculating prediction values



-------------Performing 4-fold cross validation------------------ 
